# Batch SQL Gold


In [0]:

USE IDENTIFIER(:catalog ||'.'|| :schema)


# Products

In [0]:
MERGE INTO gold_products AS T
USING (
  WITH changed_register_for_merge AS (
    SELECT 
      sha2(concat_ws('|',Product_ID,Registration_TimeStamp),256) as Product_SK,
      Product_ID,
      SKU,
      Name,
      Unit_Price,
      Moviment_Date,
      Registration_TimeStamp AS StartDate,
      Null AS EndDate,
      sha2(concat_ws('|',*),256) as hash,  --> HASH HERE DEPENDS ONLY ON THE COLUMNS OF SILVER TABLE
      Operation
    FROM silver_products
    WHERE Moviment_Date = :moviment_date

    UNION ALL

    SELECT DISTINCT T.*, Null AS Operation
    FROM ( SELECT Product_ID, Registration_TimeStamp FROM silver_products WHERE Moviment_Date = :moviment_date ) S
    LEFT JOIN gold_products T 
      ON S.Product_ID = T.Product_ID
    WHERE S.Registration_TimeStamp > T.StartDate AND S.Registration_TimeStamp < T.EndDate
  ),
  lead_lag_table AS (
  SELECT *,
    LEAD(StartDate) OVER (PARTITION BY Product_ID ORDER BY StartDate) AS EndDate1,
    LAG(EndDate) OVER (PARTITION BY Product_ID ORDER BY StartDate) AS EndDate2
  FROM changed_register_for_merge
  )
  SELECT
    Product_SK,
    Product_ID,
    SKU,
    Name,
    Unit_Price,
    Moviment_Date,
    StartDate,
    COALESCE(EndDate1, EndDate2, '9999-12-31') AS EndDate,
    hash
  FROM lead_lag_table
  WHERE Operation <> 'DELETE'
) AS S
ON T.product_ID = S.product_ID AND S.StartDate = T.StartDate

WHEN MATCHED THEN UPDATE SET T.EndDate = S.EndDate
-- Inserting:
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
21,0,0,21


In [0]:
UPDATE IDENTIFIER(:catalog ||'.control.control_tables_moviments')
SET MOVIMENT_DATE = :moviment_date
WHERE Project_Name = :project
  AND Table_name = 'gold_products'